<img src="docs/images/DSPy8.png" alt="DSPy7 Image" height="150"/>

## **DSPy**: Programming with Foundation Models

[<img align="center" src="https://colab.research.google.com/assets/colab-badge.svg" />](https://colab.research.google.com/github/stanfordnlp/dspy/blob/main/intro.ipynb)

This notebook introduces the **DSPy** framework for **Programming with Foundation Models**, i.e., language models (LMs) and retrieval models (RMs).

**DSPy** emphasizes programming over prompting. It unifies techniques for **prompting** and **fine-tuning** LMs as well as improving them with **reasoning** and **tool/retrieval augmentation**, all expressed through a _minimalistic set of Pythonic operations that compose and learn_.

**DSPy** provides **composable and declarative modules** for instructing LMs in a familiar Pythonic syntax. On top of that, **DSPy** introduces an **automatic compiler that teaches LMs** how to conduct the declarative steps in your program. The **DSPy compiler** will internally _trace_ your program and then **craft high-quality prompts for large LMs (or train automatic finetunes for small LMs)** to teach them the steps of your task.

### 0] Setting Up

As we'll start to see below, **DSPy** can routinely teach powerful models like `GPT-3.5` and local models like `T5-base` or `Llama2-13b` to be much more reliable at complex tasks. **DSPy** will compile the _same program_ into different few-shot prompts and/or finetunes for each LM.

In [1]:
import sys
import os
import dspy
from dspy.evaluate import Evaluate
from dspy.teleprompt import BootstrapFewShotWithRandomSearch
from rouge_score import rouge_scorer

In [2]:
model_name = 'gemma2:27b'
#model_name = 'qwen2.5:72b'

In [4]:
#turbo = dspy.OpenAI(model='gpt-3.5-turbo')
lm = dspy.OllamaLocal(model=model_name)
dspy.settings.configure(lm=lm)

You can build your own **DSPy programs** for various tasks, e.g., question answering, information extraction, or text-to-SQL.

Whatever the task, the general workflow is:

1. **Collect a little bit of data.** Define examples of the inputs and outputs of your program (e.g., questions and their answers). This could just be a handful of quick examples you wrote down. If large datasets exist, the more the merrier!
1. **Write your program.** Define the modules (i.e., sub-tasks) of your program and the way they should interact together to solve your task.
1. **Define some validation logic.** What makes for a good run of your program? Maybe the answers need to have a certain length or stick to a particular format? Specify the logic that checks that.
1. **Compile!** Ask **DSPy** to _compile_ your program using your data. The compiler will use your data and validation logic to optimize your program (e.g., prompts and modules) so it's efficient and effective!
1. **Iterate.** Repeat the process by improving your data, program, validation, or by using more advanced features of the **DSPy** compiler.

### Data

In [5]:
from datasets import load_dataset

multi_lexsum = load_dataset("allenai/multi_lexsum", name="v20230518")
print(multi_lexsum)

DatasetDict({
    train: Dataset({
        features: ['id', 'sources', 'sources_metadata', 'summary/long', 'summary/short', 'summary/tiny', 'case_metadata'],
        num_rows: 3177
    })
    validation: Dataset({
        features: ['id', 'sources', 'sources_metadata', 'summary/long', 'summary/short', 'summary/tiny', 'case_metadata'],
        num_rows: 454
    })
    test: Dataset({
        features: ['id', 'sources', 'sources_metadata', 'summary/long', 'summary/short', 'summary/tiny', 'case_metadata'],
        num_rows: 908
    })
})


The core data type for data in DSPy is Example. You will use Examples to represent items in your training set and test set. DSPy Examples are similar to Python dicts but have a few useful utilities. Your DSPy modules will return values of the type Prediction, which is a special sub-class of Example. When you use DSPy, you will do a lot of evaluation and optimization runs. Your individual datapoints will be of type Example.

Select 100 train examples and 20 dev examples where all 3 summaries are present

In [6]:
trainset = multi_lexsum['train'].filter(
    lambda x: x['summary/tiny'] is not None and x['summary/short'] is not None and x['summary/long'] is not None
).select(range(100))

devset = multi_lexsum['validation'].filter(
    lambda x: x['summary/tiny'] is not None and x['summary/short'] is not None and x['summary/long'] is not None
).select(range(20))

#### Create Examples

In [7]:
def join_sources(x):
    x['sources'] = ' '.join(x['sources'])
    return x

trainset = trainset.map(join_sources, batched=False)
devset = devset.map(join_sources, batched=False)

In [8]:
# Tell DSPy that the joined 'sources' field is the input. Any other fields are labels and/or metadata
trainset = [dspy.Example(
    doc=x['sources'],
    long=x['summary/long'],
    short=x['summary/short'],
    tiny=x['summary/tiny']
).with_inputs('doc') for x in trainset]

devset = [dspy.Example(
    doc=' '.join(x['sources']),
    long=x['summary/long'],
    short=x['summary/short'],
    tiny=x['summary/tiny']
).with_inputs('doc') for x in devset]

# ['doc', 'long', 'short', 'tiny']
len(trainset), len(devset)

(100, 20)

**DSPy** typically requires very minimal labeling. Whereas your pipeline may involve six or seven complex steps, you only need labels for the initial question and the final answer. **DSPy** will bootstrap any intermediate labels needed to support your pipeline. If you change your pipeline in any way, the data bootstrapped will change accordingly!

Now, let's look at some data examples.

In [9]:
train_example = trainset[1]
print(f"short: {train_example.short}")
print(f"tiny: {train_example.tiny}")

short: Two men who were arrested for trespassing on property of businesses open to the public filed a lawsuit in the U.S. District Court for the Western District of Michigan against the city of Grand Rapids, its chief of police, and two individual officers. The plaintiffs claimed that the Grand Rapids Police Department's policy and practice of arresting individuals for trespass -- without probable cause and based on general Letters of Intent to Prosecute signed by Grand Rapids businesses -- results in unreasonable searches and seizures in violation of the Fourth Amendment. The parties came to a private settlement agreement for damages and attorney's fees in late 2019. The Judge dismissed the case in early 2020.
tiny: Settlement reached in 2019 for @ACLU case on arrests for trespassing without a warning under Grand Rapids, MI's "No Trespass Letters" policy (W.D. Mich.)


In [10]:
dev_example = devset[2]
print(f"short: {dev_example.short}")
print(f"tiny: {dev_example.tiny}")

short: Pretrial detainees file lawsuit against Middlesex County in November 2015 to ameliorate the unconstitutional conditions of solitary confinement in the Middlesex County Jail. In September 2018, the parties reached a settlement agreement that restricted the maximum amount of time allowed in isolation and provides those in isolation with opportunities to interact with others.
tiny: Pretrial detainees settled this class action against Middlesex County to provide 28 hours per week of out-of-cell time and mental health screenings to people held in solitary confinement.


After loading the raw data, we'd applied `with_inputs(' '.join(x['sources']))` to each example to tell **DSPy** that our input field in each example will be just `doc`. Any other fields are labels or metadata that are not given to the system.

### Basic zero shot prompt

tiny ~ 25 words

short ~ 130 words

long ~ 650 words

Generate a {summary_type} summary of maximum {max_tokens[summary_type]} tokens of the following text:

#### Short Summary

##### Signature

In [11]:
class ShortSummSig(dspy.Signature):
    """Generate short summaries of about 130 words."""
    # input
    doc = dspy.InputField()
    # output
    short = dspy.OutputField()

In `ShortSumm`, the docstring describes the sub-task. Each `InputField` or `OutputField` can optionally contain a description `desc` too. When it's not given, it's inferred from the field's name (e.g., `question`).

Notice that there isn't anything special about this signature in **DSPy**. We can just as easily define a signature that takes a long snippet from a PDF and outputs structured information, for instance.

Anyway, now that we have a signature, let's define and use a **Predictor**. A predictor is a module that knows how to use the LM to implement a signature. Importantly, predictors can **learn** to fit their behavior to the task!

```
dspy.Example(field1=value, field2=value2, ...)
```

In [12]:
# Define the predictor.
generate_short = dspy.Predict(ShortSummSig)

# Call the predictor on a particular input
pred = generate_short(doc=dev_example.doc)

# Print the prediction
#print(f"Doc: {dev_example.doc}")
print(f"Generated short: {pred.short}")
print('')
print(f"Ground truth short: {dev_example.short}")

Generated short: This appears to be a legal document excerpt detailing court activity. Here's a breakdown:

* **Dismissal:** The case (likely a civil lawsuit, given the "cv" designation) was dismissed with prejudice against Middlesex County. This means the plaintiff cannot refile the same claim.
* **Stipulation:**  The dismissal was agreed upon by both parties (the plaintiff and Middlesex County).
* **Dates:** Key dates include:
    * 10/24/2018: Judge Peter G. Sheridan signed the dismissal order.
    * 10/25/2018: The dismissal was officially entered into the court record.
* **PACER Transaction:** This section indicates

Ground truth short: Pretrial detainees file lawsuit against Middlesex County in November 2015 to ameliorate the unconstitutional conditions of solitary confinement in the Middlesex County Jail. In September 2018, the parties reached a settlement agreement that restricted the maximum amount of time allowed in isolation and provides those in isolation with opportunities

#### Evaluation

##### Metric

In [14]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
def eval_summ(example, pred, trace=None):
    return scorer.score(example.short.lower(), pred.short.lower())['rouge1'][2]

r1 = eval_summ

##### Program

In [15]:
class ShortSummProg(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_summ = dspy.Predict(ShortSummSig)

    def forward(self, doc):
        return self.generate_summ(doc=doc)

shortsumm = ShortSummProg()

##### Evaluate

In [16]:
evaluator = Evaluate(devset=devset, num_threads=1, display_progress=True, display_table=0)

In [17]:
evaluator(shortsumm, metric=r1)

Average Metric: 4.149412227978533 / 20 


20.75

### Optimized few-shot with bootstrapped demonstrations

In [19]:
bootstrap_optimizer = BootstrapFewShotWithRandomSearch(
    max_bootstrapped_demos=8,
    max_labeled_demos=8,
    #num_candidate_programs=10,
    num_threads=8,
    metric=r1,
    #teacher_settings=dict(lm=gpt4T)
)

Going to sample between 1 and 8 traces per predictor.
Will attempt to bootstrap 16 candidate sets.


In [21]:
shortsumm_fewshot = bootstrap_optimizer.compile(shortsumm, trainset=trainset, valset=devset)

Average Metric: 4.351450206532021 / 20 


Score: 21.76 for set: [0]
New best sscore: 21.76 for seed -3
Scores so far: [21.76]
Best score: 21.76


Average Metric: 4.357405195605286 / 20 


Score: 21.79 for set: [8]
New best sscore: 21.79 for seed -2
Scores so far: [21.76, 21.79]
Best score: 21.79


  8%|▏ | 8/100 [01:18<14:57,  9.75s/it]


...

Score: 20.94 for set: [8]
Scores so far: [21.76, 21.79, 21.06, 20.67, 20.7, 20.44, 21.08, 20.84, 21.06, 20.92, 21.36, 21.77, 20.69, 21.11, 21.33, 20.77, 20.87, 20.94]
Best score: 21.79
Average of max per entry across top 1 scores: 0.21787025978026428
Average of max per entry across top 2 scores: 0.22722673687725586
Average of max per entry across top 3 scores: 0.22958757892814052
Average of max per entry across top 5 scores: 0.23457531776551419
Average of max per entry across top 8 scores: 0.23871939409538467
Average of max per entry across top 9999 scores: 0.2468081748152374


  4%|  | 4/100 [01:08<27:14, 17.03s/it]


Bootstrapped 4 full traces after 5 examples in round 0.


Average Metric: 4.211498499519415 / 20 

Score: 21.06 for set: [8]
Scores so far: [21.76, 21.79, 21.06, 20.67, 20.7, 20.44, 21.08, 20.84, 21.06, 20.92, 21.36, 21.77, 20.69, 21.11, 21.33, 20.77, 20.87, 20.94, 21.06]
Best score: 21.79
Average of max per entry across top 1 scores: 0.21787025978026428
Average of max per entry across top 2 scores: 0.22722673687725586
Average of max per entry across top 3 scores: 0.22958757892814052
Average of max per entry across top 5 scores: 0.23457531776551419
Average of max per entry across top 8 scores: 0.23871939409538467
Average of max per entry across top 9999 scores: 0.2468081748152374
19 candidate programs found.
